In [2]:
# 31319413,19752556中发现"ALZHEI-MER'S DISEASE"划分到了'Parkinson Disease'，需要处理！

In [1]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [2]:
import pickle
import json
import time
import datetime
import copy

In [3]:
now=datetime.datetime.now()

In [4]:
nowtime=now.strftime('%Y/%m/%d')

In [5]:
now_time=now.strftime('%Y_%m_%d')

In [6]:
print(nowtime)

2023/03/23


In [7]:
time_without_day=now.strftime('%Y/%m')

In [8]:
time_without_day

'2023/03'

In [9]:
### today 2022_07 FORMAT
### now.strftime('%Y_%m')

In [10]:
### 2022/07/30 FORMAT
def cal_time(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    # 防止月份出界
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [11]:
### 2022_07_30 FORMAT
def cal_time2(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    # 防止月份出界
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y_%m_%d')
    return time

In [12]:
### 2022_07 FORMAT
def cal_time3(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    # 防止月份出界
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y_%m')
    return time

In [13]:
import pymongo 

In [14]:
from pymongo import MongoClient

In [15]:
client = MongoClient(username="hald",password="XXX")

In [16]:
db = client.hald

In [20]:
raw_pick=pickle.load(open('results/step1/raw_pick.pkl','rb'))

In [21]:
raw_pick_pubtator=pickle.load(open('results/step1/raw_pick_pubtator.pkl','rb'))

In [22]:
pubtator_raw=pickle.load(open('results/step1/pubtator_raw.pkl','rb'))

In [23]:
pubtator_raw_dict2AB={}
for i in pubtator_raw:
    pubtator_raw_dict2AB[i.get('id')]=i.get('AB')

# Gene_raw

In [24]:
collections_Gene_raw=db.Gene_raw

In [25]:
rawpickwithAB={}
for i in raw_pick_pubtator:
    rawpickwithAB[i.get('PMID')]=i

In [26]:
pubtator_gene=[]
for i in pubtator_raw:
    if i.get('type')=='Gene':
        pubtator_gene.append(i)

In [27]:
#7246
len(pubtator_gene)

4849

In [28]:
import pandas as pd
import json
import re

In [29]:
Gene2ncbi=pd.read_csv('../ncbi/Gene_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [30]:
ncbi2Gene_dict=dict(zip(Gene2ncbi['NCBI Gene ID(supplied by NCBI)'],Gene2ncbi['Approved symbol'] ))

In [31]:
error=[]
pubtator_genes=[]
for i in pubtator_gene:
    if ncbi2Gene_dict.get(i.get('identifier')):
        i['entity']=ncbi2Gene_dict.get(i.get('identifier'))
        pubtator_genes.append(i)
    else:
        error.append(i)

In [32]:
len(pubtator_genes)

3939

In [33]:
Genelist=pickle.load(open('../dict/Genedict_list.pkl','rb'))

In [34]:
Gene_list={}
for i in Genelist:
    Gene_list[i.lower()]=i.lower()

In [35]:
filtered_pubtator_genes=[]
drop_genes=[]
for i in pubtator_genes:
    if Gene_list.get(i.get('text').lower()):
        filtered_pubtator_genes.append(i)
    else:
        drop_genes.append(i)

In [36]:
len(drop_genes)

970

In [37]:
len(filtered_pubtator_genes)

2969

In [38]:
filtered_pubtator_genes[10]

{'id': '36884388',
 'AB': 'Osteoarthritis (OA) pathogenesis is associated with reduced chondrocyte homeostasis and increased levels of cartilage cellular senescence. Chondrosenescence is the development of cartilage senescence that increases with aging joints and disrupts chondrocyte homeostasis and is associated with OA. Adenosine A2A receptor (A2AR) activation in cartilage via intra-articular injection of liposomal A2AR agonist, liposomal-CGS21680, leads to cartilage regeneration in vivo and chondrocyte homeostasis. A2AR knockout mice develop early OA isolated chondrocytes demonstrate upregulated expression of cellular senescence and aging-associated genes. Based on these observations, we hypothesized that A2AR activation would ameliorate cartilage senescence. We found that A2AR stimulation of chondrocytes reduced beta-galactosidase staining and regulated levels and cell localization of common senescence mediators p21 and p16 in vitro in the human TC28a2 chondrocyte cell line. In viv

In [39]:
for i in filtered_pubtator_genes:
    i['ner']='HALD_Gene'
    i['PMID']=i.get('id')
    i['word']=i.get('text')

In [40]:
import spacy
import scispacy

import pandas as pd
import requests 
from collections import Counter
from tqdm.notebook import tqdm
tqdm.pandas()
import json
import ast
from math import ceil
from itertools import combinations, permutations
import en_ner_bc5cdr_md, en_ner_bionlp13cg_md, en_ner_jnlpba_md
from spacy.tokenizer import Tokenizer
import string
from itertools import permutations

/root/miniconda3/envs/hald/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
ner_bc5c = en_ner_bc5cdr_md.load()
ner_bionl = en_ner_bionlp13cg_md.load()
ner_jnlp = en_ner_jnlpba_md.load()

In [42]:
def get_entities(ti, model, keep=[]):
    doc = model(ti)
    if len(keep) > 0:
        return [(x.text.replace('\n', ''), x.label_) for x in doc.ents if str(x.label_) in keep]
    return [(x.text.replace('\n', ''), x.label_) for x in doc.ents ]

In [43]:
raw_pick_pubtator_pmid=[]
for i in raw_pick_pubtator:
    if i.get('AB'):
        raw_pick_pubtator_pmid.append(i.get('PMID'))
    else:
        pass

In [44]:
filtered_pubtator_genes[0]

{'id': '36890535',
 'AB': "BACKGROUND: Anti-vascular endothelial growth factor is a medicine administered intravitreally by an injection to maintain visual acuity in patients with a variety of retinal diseases. The demand for this treatment has grown considerably in the westernized world the last two decades and will continue to increase due to an aging population. Because of the high volume, injections seize enormous resources and represent high costs for both hospitals and society. Task-shifting of injections from physicians to nurses may be a means to reduce such costs, however the magnitude of possible savings has been poorly investigated. To this end we investigated changes in the hospital costs per injection, six-year cost projections of physician- versus nurse-administered injections for a Norwegian tertiary hospital and we compared the societal costs per patient per year. METHODS: Patients (n = 318) were randomized to either physician- or nurse administered injections, and data

prepare_for_scispacy={}
for i in filtered_pubtator_genes:
    if not filtered_pubtator_genes.get(i.get('PMID')):
        filtered_pubtator_genes[i.get('PMID')]=i

In [45]:
# 186s
import datetime
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
def work(i,finallist_tags,lock):
    # lock.acquire()
    finallist_tags.append({'PMID':i.get('PMID'), 'AB':i.get('AB'), 'sentence':i.get('sentence'), 'sentenceIndex': i.get('sentenceIndex'),\
                          'sentenceSize':i.get('sentenceSize'),'word': i.get('word'),'identifier':i.get('identifier'),'type':i.get('type'),\
                           'entity':i.get('entity'),'jnlpba_tags':get_entities(i.get('sentence'), model=ner_jnlp, keep=['DNA','RNA','PROTEIN']),\
                          'bc5cdr_tags':get_entities(i.get('sentence'), model=ner_bc5c),\
                          'bionlp13cg_tags':get_entities(i.get('sentence'), model=ner_bionl, \
                          keep=['GENE_OR_GENE_PRODUCT'])})
    # lock.release()
    # bc5cdr_tags: DISEASE, CHEMICAL
    
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(60)
    manager=Manager()
    finallist_tags=manager.list()
    lock=manager.Lock()
    for i in filtered_pubtator_genes:
        p.apply_async(work, args=(i,finallist_tags,lock))

    print('Waiting for all subprocesses done...')
    
    p.close()
    p.join()
    print('All subprocesses done.')

    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 75997.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 30s


In [46]:
Gene_tags=list(finallist_tags)

In [47]:
#4538
len(Gene_tags)

2969

In [48]:
import copy

In [49]:
scispicy_Gene=copy.deepcopy(Gene_tags)

In [50]:
for i in scispicy_Gene:
    i['ScispaCy']=[]
    if i.get('jnlpba_tags'):
        for j in i.get('jnlpba_tags'):
            if j[1]=='DNA' or j[1]=='PROTEIN':
                i['ScispaCy']+=[j[0]]
    if i.get('bionlp13cg_tags'):
        for k in i.get('bionlp13cg_tags'):
            i['ScispaCy']+=[k[0]]
    i['ScispaCy']=list(set(i.get('ScispaCy')))

In [51]:
scicpacy_Genes=[]
not_scicpacy_Genes=[]
for i in scispicy_Gene:
    if i.get('ScispaCy'):
        scicpacy_Genes.append(i)
    else:
        not_scicpacy_Genes.append(i)

In [52]:
# 4123
len(scicpacy_Genes)

2761

In [53]:
len(not_scicpacy_Genes)

208

In [54]:
not_scicpacy=[]
for i in not_scicpacy_Genes:
    not_scicpacy.append({'PMID':i.get('PMID'),'word':i.get('word'),'sentence':i.get('sentence')})

In [55]:
not_scicpacy[-10:-1]

[{'PMID': '34541986',
  'word': 'ACE',
  'sentence': 'RESULTS: We found that: (1) people who experienced ACEs had trajectories of depressive symptoms in late life that were higher than people not having these experiences, (2) people experiencing two or more ACEs were more likely to report fear and distress when Hurricane Sandy hit than people experiencing either one or no adverse childhood experiences, and (3) while both ACE exposure and peri-traumatic stress were associated with trajectories having higher levels of depressive symptoms, the risk associated with ACEs (especially multiple ACEs) was greater.'},
 {'PMID': '34375545',
  'word': 'aid',
  'sentence': "CONCLUSION: Research is needed to investigate the benefits of extensive gait aid training in older people with mobility problems, including those with dementia or high falls risk.IMPLICATIONS FOR REHABILITATIONThere is little evidence currently addressing the benefits of gait aid prescription on gait and associated outcomes in o

In [56]:
scicpacy_Genes_final=[]
drop_scispacy=[]
for i in scicpacy_Genes:
    pattern=i.get('word').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                              "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
    for j in i.get('ScispaCy'):
        if i.get('word')==j or re.search(pattern, j):
            scicpacy_Genes_final.append(i)
            break
    else:
        drop_scispacy.append(i)

In [57]:
# 3701
len(scicpacy_Genes_final)

2492

In [58]:
for i in scicpacy_Genes_final:
    if re.search('-',i.get('word')):
        print(i)
        break

{'PMID': '36874925', 'AB': 'Aging slows down the mechanisms behind skeletal muscle weakening and mobility. Increases in inflammation brought on by aging may contribute to some characteristics of sarcopenia. As a result of population aging worldwide, sarcopenia, an age-related disease, has become a huge burden on both individuals and society as a whole. The study of the morbidity mechanism and available sarcopenia treatments has received more attention. The inflammatory response may be one of the most important methods behind the pathophysiology of sarcopenia in the aged, according to the background of the study. This anti-inflammatory cytokine inhibits the ability of human monocytes and macrophages to induce inflammation as well as the production of cytokines like IL-6. Here, we investigate the association between sarcopenia and interleukin-17 (IL-17), an inflammatory cytokine in the aged. There were 262 subjects aged 61-90 years who were screened for sarcopenia in Hainan General Hospi

In [59]:
scicpacy_Genes_final[0]

{'PMID': '36890535',
 'AB': "BACKGROUND: Anti-vascular endothelial growth factor is a medicine administered intravitreally by an injection to maintain visual acuity in patients with a variety of retinal diseases. The demand for this treatment has grown considerably in the westernized world the last two decades and will continue to increase due to an aging population. Because of the high volume, injections seize enormous resources and represent high costs for both hospitals and society. Task-shifting of injections from physicians to nurses may be a means to reduce such costs, however the magnitude of possible savings has been poorly investigated. To this end we investigated changes in the hospital costs per injection, six-year cost projections of physician- versus nurse-administered injections for a Norwegian tertiary hospital and we compared the societal costs per patient per year. METHODS: Patients (n = 318) were randomized to either physician- or nurse administered injections, and da

In [60]:
len(drop_scispacy)

269

In [61]:
scicpacy_pmids=[]
for i in scicpacy_Genes_final:
    if i.get('PMID') not in scicpacy_pmids:
        scicpacy_pmids.append(i.get('PMID'))

In [62]:
len(scicpacy_pmids)

532

In [63]:
def query_raw_id(ids, url="https://bern.korea.ac.kr/pubmed/"):
    return requests.get(url + ','.join(ids), verify=False).json()

now_time='2022_12_01'

In [64]:
os.makedirs('../bern/'+now_time)

In [65]:
f=open('../bern/'+now_time+'/download.txt','a')
bern_download_id=1
for j in range(0,len(scicpacy_pmids),5):
    f.write('wget https://bern.korea.ac.kr/pubmed/'+','.join(scicpacy_pmids[j:j+5])+' --no-check-certificate -O '+'../bern/'+now_time+'/bern'+str(bern_download_id)+'.json \n')
    bern_download_id+=1
f.close()

In [66]:
f=open('../bern/'+now_time+'/run.sh','w')
f.write("while read rows; do $rows  ; done < ../bern/"+now_time+'/download.txt')
f.close()

In [67]:
print("nohup bash "+'../bern/'+now_time+'/run.sh > '+'../bern/'+now.strftime('%Y_%m_%d')+'/bern.log &')

nohup bash ../bern/2023_03_14/run.sh > ../bern/2023_03_14/bern.log &


## RUN BERN ON LINUX!!!!!!
## Check the file size!!!!!!

In [66]:
'''
%%capture
os.system("bash "+'../bern/'+now.strftime('%Y_%m_%d')+'/run.sh > '+'../bern/'+now.strftime('%Y_%m_%d')+'/bern.log')
'''

'\n%%capture\nos.system("bash "+\'../bern/\'+now.strftime(\'%Y_%m_%d\')+\'/run.sh > \'+\'../bern/\'+now.strftime(\'%Y_%m_%d\')+\'/bern.log\')\n'

In [68]:
all_json=[]
for i in os.listdir('../bern/'+now_time+'/'):
    if i.split('.')[-1] == 'json':
        all_json+=json.load(open('../bern/'+now_time+'/'+i))

In [69]:
allbern=copy.deepcopy(all_json)

In [70]:
len(allbern)

532

In [71]:
bern_json={}
for i in allbern:
    bern_value=[]
    if i.get('denotations'):
        for j in i.get('denotations'):
            if j.get('obj')=='gene' or j.get('obj')=='protein':
                bern_value.append(i.get('text')[j.get('span').get('begin'):j.get('span').get('end')])
    bern_json[i.get('sourceid')]=bern_value    

In [72]:
len(bern_json)

532

In [73]:
count=0
filtered_bern_json={}
for i,j in bern_json.items():
    if j:
        count+=1
        filtered_bern_json[i]=j
print(count)

521


In [74]:
list(filtered_bern_json.items())[0]

('36845012',
 ['aryl hydrocarbon receptor nuclear translocator-like',
  'ARNTL',
  'BTG antiproliferation factor 2',
  'BTG2',
  'motif chemokine ligand 10',
  'CXCL10',
  'chitinase 3-like 1',
  'CHI3L1',
  'immediate early response 3',
  'IER3',
  'Fos proto-oncogene',
  'AP-1 transcription factor subunit',
  'FOS',
  'peroxisome proliferative activated receptor, gamma, coactivator 1 alpha',
  'PPARGC1A',
  'ARNTL',
  'BTG2',
  'CXCL10',
  'CHI3L1',
  'IER3',
  'FOS',
  'PPARGC1A',
  'IRI'])

In [75]:
SciSpacy_raw=copy.deepcopy(scicpacy_Genes_final)

In [76]:
len(SciSpacy_raw)

2492

In [77]:
SciSpacy_raw_dict={}
for i in SciSpacy_raw:
    SciSpacy_raw_dict[i.get('PMID')]=SciSpacy_raw_dict.get(i.get('PMID'),[])+[i]

In [78]:
len(SciSpacy_raw_dict)

532

In [79]:
cs=0
intersec={}
dropout=[]
for i,j in filtered_bern_json.items():
    if SciSpacy_raw_dict.get(i):
        midd=[]
        for k in SciSpacy_raw_dict.get(i):
            if k.get('word') in j:
                k['bern']=j
                midd.append(k)
            else:
                for m in j:
                    pattern=k.get('word').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
                    if re.search(pattern, m):
                        k['bern']=j
                        midd.append(k)
                        break
                else:
                    dropout.append(k)
        intersec[i]=midd
        

In [80]:
len(intersec)

521

In [81]:
len(dropout)

73

In [82]:
final_Gene_raw=[]
for i in intersec.values():
    final_Gene_raw+=i

In [83]:
len(final_Gene_raw)

2398

In [84]:
Genes_inte=copy.deepcopy(final_Gene_raw)

In [85]:
Gene2blank={}
for i in Genes_inte:
    Gene2blank[i.get('entity')]=''

In [86]:
len(Gene2blank)

569

In [87]:
Gene2ncbi=pd.read_csv('../ncbi/Gene_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [88]:
Gene2ncbi_dict=dict(zip(Gene2ncbi['Approved symbol'], Gene2ncbi['NCBI Gene ID(supplied by NCBI)']))

In [89]:
Gene_converter=pickle.load(open('../ncbi/Gene_converter.pkl','rb'))

In [90]:
len(Gene_converter)

4854

In [91]:
Genes_inte_copy=copy.deepcopy(Genes_inte)

In [92]:
rawpickwithAB={}
for raw in raw_pick_pubtator:
    if raw.get('AB'):
        rawpickwithAB[raw.get('PMID')]=raw

In [93]:
Gene_deduplicated=set()
Gene_deduplicated_count=0
Gene_list_raw=[]

for i in Genes_inte_copy:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['JT']=rawpickwithAB.get(k).get('JT')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['target']=i.get('word')
    i['type']='Gene'
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Gene_deduplicated:
        Gene_deduplicated.add(tuples)
        Gene_list_raw.append(i)
    else:
        Gene_deduplicated_count+=1
        continue

In [94]:
len(Gene_list_raw)

2196

In [95]:
Gene2blank={}
for i in Gene_list_raw:
    Gene2blank[i.get('entity')]=''

In [96]:
len(Gene2blank)

569

In [101]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_Gene_info(i,final_Genes_inte):
    ncbiid=Gene2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    # 'Also_known_as' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:',''),
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_Genes_inte[i]=ncbisummary
    # print(ncbisummary)
    time.sleep(0.5)

In [102]:
judge=False
p = Pool(10)
manager=Manager()
final_Genes_inte = manager.dict()
for i in Gene2blank.keys():
    if not Gene_converter.get(i):
        p.apply_async(get_Gene_info, args=(i,final_Genes_inte))
        judge=True
p.close()
p.join()
if judge:
    Gene_converter.update(dict(final_Genes_inte))
    pickle.dump(Gene_converter, open('../ncbi/Gene_converter.pkl','wb'))

In [109]:
judge

True

In [103]:
# 4854
len(Gene_converter)

4953

In [104]:
Gene_converter.get('CLPS')

{'Official_Symbol': 'CLPS',
 'Official_Full_Name': 'colipase',
 'HGNC_ID': 'HGNC:HGNC:2085',
 'HGNC_URL': 'https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/HGNC:2085',
 'Ensembl_ID': 'Ensembl:ENSG00000137392',
 'Ensembl_URL': 'http://www.ensembl.org/id/ENSG00000137392',
 'MIM_ID': 'MIM:120105;',
 'MIM_URL': 'https://www.ncbi.nlm.nih.gov/omim/120105',
 'AllianceGenome_ID': 'AllianceGenome:HGNC:2085',
 'AllianceGenome_URL': 'https://www.alliancegenome.org/gene/HGNC:2085',
 'NCBI_ID': '1208',
 'NCBI_URL': 'https://www.ncbi.nlm.nih.gov/gene/1208',
 'Also_known_as': '',
 'Summary': 'The protein encoded by this gene is a cofactor needed by pancreatic lipase for efficient dietary lipid hydrolysis. It binds to the C-terminal, non-catalytic domain of lipase, thereby stabilizing an active conformation and considerably increasing the overall hydrophobic binding site. The gene product allows lipase to anchor noncovalently to the surface of lipid micelles, counteracting the destabilizin

In [105]:
for i in range(len(Gene_list_raw)):
    Gene_list_raw[i]=dict(Gene_list_raw[i],**Gene_converter.get(Gene_list_raw[i].get('entity')))

In [106]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in Gene_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [107]:
for i in Gene_list_raw:
    i['addtime']=now_time

In [108]:
Gene_raw=copy.deepcopy(Gene_list_raw)

In [110]:
len(Gene_raw)

2196

collections_Gene_raw.remove({'addtime':now_time})

collections_Gene_raw.remove({'addtime':now_time})

In [112]:
if len(Gene_raw)!=0:
    collections_Gene_raw.insert_many(Gene_raw)

collections_Gene_raw.remove({'addtime':now_time})

for i in Gene_raw_list:
    if i.get('Expression'):
        del i['Expression']
    if Gene_converter.get(i.get('entity')).get('Summary'):
        i['Summary']=Gene_converter.get(i.get('entity')).get('Summary')
    else:
        i['Summary']=''
    i['ner']='HALD_Gene'

for i in Gene_raw_list:
    if i.get('addtime')=='2023_02_25':
        print(i)
        break

len(Gene_raw_list)

collections_Gene_raw.drop()

collections_Gene_raw.insert_many(Gene_raw_list)

# Gene_all

In [113]:
Gene_raw_list=list(collections_Gene_raw.find())

In [114]:
for i in Gene_raw_list:
    i['type']='Gene'
    i['JT']=[i.get('JT')]
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]

In [115]:
Gene_mid={}
for i in Gene_raw_list:
    if not Gene_mid.get(i.get("entity")):
        Gene_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Gene_mid.get(i.get('entity')).get('PMID'):

            ind=Gene_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Gene_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Gene_mid[i.get('entity')]['sentence'][ind] = Gene_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Gene_mid[i.get('entity')]['target'][ind] = Gene_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Gene_mid[i.get("entity")]["PMID"]=Gene_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Gene_mid[i.get("entity")]["sentence"]=Gene_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Gene_mid[i.get("entity")]["AB"]=Gene_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Gene_mid[i.get("entity")]["DP"]=Gene_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Gene_mid[i.get("entity")]["TI"]=Gene_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Gene_mid[i.get("entity")]["target"]=Gene_mid.get(i.get("entity")).get("target")+i.get("target")
            Gene_mid[i.get("entity")]["date"]=Gene_mid.get(i.get("entity")).get("date")+i.get("date")
            Gene_mid[i.get("entity")]["year"]=Gene_mid.get(i.get("entity")).get("year")+i.get("year")
            Gene_mid[i.get("entity")]["JT"]=Gene_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [116]:
Gene_output_json_final=list(Gene_mid.values())

In [117]:
# 4854
len(Gene_output_json_final)

4953

now_time='2022_12_09'

In [118]:
for i in Gene_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [119]:
Gene_all=copy.deepcopy(Gene_output_json_final)

# RNA_raw

In [120]:
collections_RNA_raw=db.RNA_raw

collections_RNA_raw.remove({'addtime':now_time})

In [121]:
names=locals()

In [122]:
raw_pick=pickle.load(open('results/step1/raw_pick.pkl','rb'))

In [123]:
raw_pick_pubtator=pickle.load(open('results/step1/raw_pick_pubtator.pkl','rb'))

In [124]:
pubtator_raw=pickle.load(open('results/step1/pubtator_raw.pkl','rb'))

In [149]:
pubtator_gene=[]
for i in pubtator_raw:
    if i.get('type')=='Gene':
        pubtator_gene.append(i)

In [125]:
os.makedirs('../microRNA_lncRNA_rules/'+now_time)

In [126]:
with open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt','a') as f:
    for i in raw_pick_pubtator:
        f.write(i.get('AB')+'\n'+'\n')

In [127]:
with open('../microRNA_lncRNA_rules/'+now_time+'/run.sh','w') as f:
    f.write('''
#!/bin/bash\n
start_time=$(date +%s)
java -cp "../stanza_corenlp/*" -Xmx500g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators 'tokenize,ssplit,regexner' -file "../microRNA_lncRNA_rules/'''
+now_time+'''/allsentences.txt" -regexner.mapping ../microRNA_lncRNA_rules/jg-regexner.txt -regexner.ignorecase true -tokenize.options "splitHyphenated=false" --outputFormat json --outputDirectory ../microRNA_lncRNA_rules/'''+now_time+'''/  
end_time=$(date +%s)
cost_time=$[ $end_time-$start_time ]
echo "build kernel time is $(($cost_time/60))min $(($cost_time%60))s"
    
    
    ''')

In [128]:
import os

In [129]:
os.system("bash ../microRNA_lncRNA_rules/"+now_time+"/run.sh > ../microRNA_lncRNA_rules/"+now_time+"/run.log")

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator regexner
[main] INFO edu.stanford.nlp.pipeline.TokensRegexNERAnnotator - regexner: Read 21 unique entries out of 22 from ../microRNA_lncRNA_rules/jg-regexner.txt, 0 TokensRegex patterns.

Processing file /root/hald_autoupdate/2023_03_13/../microRNA_lncRNA_rules/2023_03_14/allsentences.txt ... writing to /root/hald_autoupdate/microRNA_lncRNA_rules/2023_03_14/allsentences.txt.json
Annotating file /root/hald_autoupdate/2023_03_13/../microRNA_lncRNA_rules/2023_03_14/allsentences.txt ... done [143.7 sec].

Annotation pipeline timing information:
TokenizerAnnotator: 3.3 sec.
WordsToSentencesAnnotator: 1.7 sec.
TokensRegexNERAnnotator: 138.6 sec.
TOTAL: 143.7 sec. for 1019840 tokens at 7098.1 tokens/sec.
Pipeline setup: 0.5 sec.
Total time for StanfordCoreNLP pi

0

In [130]:
jsonfile={}
json_file=json.load(open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt.json',"r"))
jsonfile["sentences"]=json_file.get("sentences")

In [131]:
import time
import datetime
words=[]
entities_dict_list=[]
starttime=datetime.datetime.now()
l=0
HALDnames=[]
start=0

for HALDname in ["LNCRNA","MICRORNA"]:
    HALDnames.append("HALD_"+HALDname)
    

f=open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt')
sentences=f.read()
for i in jsonfile.get("sentences"):
    for HALDname in HALDnames:
        judge=True
        size=len(i.get("tokens"))
        for k,j in enumerate(i.get("tokens")):
            if j.get("ner"):
                if j.get("ner")==HALDname:
                    combination=j.get('word')
                    words.append(combination)
                    entities_dict={}
                    entities_dict["word"]=combination
                    entities_dict["wordBegin"]=j.get("characterOffsetBegin")+start
                    entities_dict["wordEnd"]=j.get("characterOffsetEnd")+start
                    entities_dict["sentenceBegin"]=i.get("tokens")[0].get("characterOffsetBegin")+start
                    entities_dict["sentenceEnd"]=i.get("tokens")[-1].get("characterOffsetEnd")+start
                    entities_dict["sentence"]=sentences[i.get("tokens")[0].get("characterOffsetBegin"):i.get("tokens")[-1].get("characterOffsetEnd")]
                    entities_dict["ner"]=HALDname
                    entities_dict_list.append(entities_dict)


                    if judge:
                        names['l'+HALDname]=k+1
                        judge=False
                    if names['l'+HALDname]<k:
                        names['l'+HALDname]=k+1

                    if k != names['l'+HALDname] -1:
                        continue


                    if names['l'+HALDname]<size:
                        if i.get("tokens")[names['l'+HALDname]].get("ner"):
                            if i.get("tokens")[names['l'+HALDname]].get("ner")==HALDname:
                                while i.get("tokens")[names['l'+HALDname]].get("ner")==HALDname:
                                    combination+=" "+i.get("tokens")[names['l'+HALDname]].get("word")
                                    names['l'+HALDname]+=1
                                    if names['l'+HALDname]>=size:
                                        break
                                names['l'+HALDname]+=1
                                words.append(combination)
                                entities_dict={}
                                entities_dict["word"]=combination
                                entities_dict["wordBegin"]=j.get("characterOffsetBegin")+start
                                entities_dict["wordEnd"]=j.get("characterOffsetEnd")+start
                                entities_dict["sentenceBegin"]=i.get("tokens")[0].get("characterOffsetBegin")+start
                                entities_dict["sentenceEnd"]=i.get("tokens")[-1].get("characterOffsetEnd")+start
                                entities_dict["sentence"]=sentences[i.get("tokens")[0].get("characterOffsetBegin"):i.get("tokens")[-1].get("characterOffsetEnd")]
                                entities_dict["ner"]=HALDname
                                entities_dict_list.append(entities_dict)

f.close()
start+=len(sentences)
                    
endtime=datetime.datetime.now()
print("The running time is "+str((endtime-starttime).seconds)+"s")

The running time is 0s


In [132]:
len(entities_dict_list)

78

In [133]:
entities_dict_list[2]

{'word': 'miR-525-5p',
 'wordBegin': 500750,
 'wordEnd': 500760,
 'sentenceBegin': 500707,
 'sentenceEnd': 501017,
 'sentence': 'In summary, we identified a new circFNDC3B/miR-525-5p/HO-1 signaling pathway that may act to relieve OA by alleviating oxidative stress and regulating the NF-kappaB pathway, resulting in the protection of the ECM in human chondrocytes, highlighting it as a potential therapeutic target for the treatment of OA.',
 'ner': 'HALD_MICRORNA'}

In [134]:
length2pmid=[]
start=0
for i in raw_pick_pubtator:
    mid=[]
    mid.append(start)
    mid.append(i.get('PMID'))
    length2pmid.append(mid)
    start+=len(i.get('AB'))+2

In [135]:
len(length2pmid)

3400

In [136]:
length2pmid[10]

[21817, '36892472']

In [137]:
k=0
for i in entities_dict_list:
    for j in range(k,len(length2pmid)):
        if i.get('wordBegin')<length2pmid[j][0]:
            i['PMID']=length2pmid[j-1][1]
            i['AB']=pubtator_raw_dict2AB.get(i.get('PMID'))
            k=j-1
            break
    else:
        i['PMID']=length2pmid[-1][1]
        i['AB']=pubtator_raw_dict2AB.get(i.get('PMID'))

In [138]:
entities_dict_list[10]

{'word': 'miR143',
 'wordBegin': 708285,
 'wordEnd': 708291,
 'sentenceBegin': 708216,
 'sentenceEnd': 708338,
 'sentence': 'Expression levels of well-known (Amh, Gdf9, and Inhibins) and novel (miR143/145 and Gas5) biomarkers of POA were assessed.',
 'ner': 'HALD_MICRORNA',
 'PMID': '36769379',
 'AB': 'Although the imbalance of circulating levels of Thyroid Hormones (THs) affects female fertility in vertebrates, its involvement in the promotion of Premature Ovarian Aging (POA) is debated. Therefore, altered synthesis of THs in both thyroid and ovary can be a trait of POA. We investigated the relationship between abnormal TH signaling, dysthyroidism, and POA in evolutionary distant vertebrates: from zebrafish to humans. Ovarian T3 signaling/metabolism was evaluated by measuring T3 levels, T3 responsive transcript, and protein levels along with transcripts governing T3 availability (deiodinases) and signaling (TH receptors) in distinct models of POA depending on genetic background and en

In [139]:
import pandas as pd

In [140]:
RNA2ncbi=pd.read_csv('../ncbi/RNA_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [141]:
RNA2approved_dict=dict(zip(RNA2ncbi['NCBI Gene ID(supplied by NCBI)'],RNA2ncbi['Approved symbol'] ))

In [142]:
RNA2ncbi_dict=dict(zip(RNA2ncbi['Approved symbol'], RNA2ncbi['NCBI Gene ID(supplied by NCBI)']))

In [150]:
# add entity
error_RNA=[]
pubtator_RNAs=[]
for i in pubtator_gene:
    if RNA2approved_dict.get(i.get('identifier')):
        i['entity']=RNA2approved_dict.get(i.get('identifier'))
        pubtator_RNAs.append(i)
    else:
        error_RNA.append(i)

In [151]:
# 7165
len(error_RNA)

4794

In [153]:
len(pubtator_RNAs)

55

In [154]:
names=locals()
HALDnames=[]
for HALDname in ["CLUSTERRNA","LNCRNA","MICRORNA","MISCRNA","RIBOSOMALRNA","SMALL_NUCLEARRNA","SMALL_NUCLEOLARRNA","TRANSFERRNA","VAULTRNA","YRNA"]:
    HALDnames.append("HALD_"+HALDname)

for name in ["clusterRNA","lncRNA","microRNA","miscRNA","ribosomalRNA","small_nuclearRNA","small_nucleolarRNA","transferRNA","vaultRNA","yRNA"]:
    file="../dict/"+name+"/"+name+"dict.json"
    with open(file,"r") as f:
        jsons=json.load(f)
        json_lower={}
        for i,j in jsons.items():
            json_lower[i.lower()]=j
        names[name+"_dict"]=json_lower

In [155]:
pattern=['([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-]?\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]?[\-]?\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]? \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?([Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-]?\d+)([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] \d+([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?([Mm][Ii][Rr][Nn]?[\-]?\d+)([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]? \d+([a-z][/]?)+',
'[Hh][Ss][Aa][\-][Mm][Ii][Rr][Nn]?[\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa][\-][Mm][Ii][Rr][Nn]?[\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Rr][Nn]?[Ll][Ee][Tt]7[A-Ia-i][123]?',
'([Mm][Ii][Rr][Nn]?[Ll][Ee][Tt]7)([A-Ia-i][/]?)+',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-][Ll][Ee][Tt][\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] [Ll][Ee][Tt] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Rr][Nn]?[\-][Ll][Ee][Tt][\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Rr][Nn]? [Ll][Ee][Tt] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-][Ll][Ee][Tt][\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] [Ll][Ee][Tt] \d+([a-z][/]?)+',
'([Mm][Ii][Rr][Nn]?[\-][Ll][Ee][Tt][\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Rr][Nn]? [Ll][Ee][Tt] \d+([a-z][/]?)+']

In [156]:
# add RNA type
filtered_pubtator_RNAs=[]
drop_RNAs=[]
import re
midcount1=0
midcount2=0
for i in pubtator_RNAs:
    for name in ["clusterRNA","lncRNA","microRNA","miscRNA","ribosomalRNA","small_nuclearRNA","small_nucleolarRNA","transferRNA","vaultRNA","yRNA"]:
        if names[name+"_dict"].get(i.get('text').lower()):
            i['ner']="HALD_"+name.upper()
            i['type']="RNA"
            i['specific_type']=name.upper()
            filtered_pubtator_RNAs.append(i)
            break
    else:
        for j in pattern:
            if re.search(j,i.get('text').lower()):
                midcount1+=1
                i['ner']="HALD_MICRORNA"
                i['type']="RNA"
                i['specific_type']="MICRORNA"
                filtered_pubtator_RNAs.append(i)
                break
            elif re.search(r'linc\d{5}',i.get('text').lower()):
                midcount2+=1
                i['ner']="HALD_LINCRNA"
                i['type']="RNA"
                i['specific_type']="LINCRNA"
                filtered_pubtator_RNAs.append(i)
                break
        else:
            drop_RNAs.append(i)

In [157]:
print(midcount1)
print(midcount2)

1
0


In [158]:
len(drop_RNAs)

4

In [159]:
filtered_pubtator_genes_pmid2word={}
for i in filtered_pubtator_RNAs:
    filtered_pubtator_genes_pmid2word[i.get('id')]=filtered_pubtator_genes_pmid2word.get(i.get('id'),[])+[i.get('text')]

In [160]:
len(filtered_pubtator_genes_pmid2word)

12

In [161]:
# based on dict. Whether in pubtator
RNA_entities_list=[]
for i in entities_dict_list:
    if filtered_pubtator_genes_pmid2word.get(i.get('PMID')):
        if i.get('word') not in filtered_pubtator_genes_pmid2word.get(i.get('PMID')):
            RNA_entities_list.append(i)
    else:
        RNA_entities_list.append(i)

In [162]:
len(RNA_entities_list)

48

In [163]:
len(entities_dict_list)

78

In [164]:
RNA_entities_list[11]

{'word': 'hsa-miR-223-3p',
 'wordBegin': 1064285,
 'wordEnd': 1064299,
 'sentenceBegin': 1064245,
 'sentenceEnd': 1064393,
 'sentence': 'Our previous miRNA profiling identified hsa-miR-223-3p as a dysregulated miRNA in the DKD samples, which may serve as a biomarker for DKD diagnosis.',
 'ner': 'HALD_MICRORNA',
 'PMID': '36731227',
 'AB': 'Diabetic kidney disease (DKD), the most pervasive complication in diabetic patients, has become a major health threat to the aging population. Our previous miRNA profiling identified hsa-miR-223-3p as a dysregulated miRNA in the DKD samples, which may serve as a biomarker for DKD diagnosis. However, the specific mechanism of miR-223-3p in the pathogenesis of DKD remains to be elucidated. In this study, we first verified that miR-223-3p level was significantly decreased in the in vitro cell model and in vivo db/db DKD model, accompanied with endothelial cell damage. Importantly, inhibiting the expression of miR-223-3p exacerbated high-glucose induced 

In [165]:
import re
for i in RNA_entities_list:
    i['type']='RNA'
    i['specific_type']=i.get('ner')[5:]

In [166]:
RNAdict=json.load(open('../dict/RNAdict.json','r'))

In [167]:
# 考虑了大小写引起的可能不是一个实体的问题
# 验证后发现不会发生
RNAdict_lower={}
for i,j in RNAdict.items():
    RNAdict_lower[i.lower()]=RNAdict_lower.get(i.lower(),[])+j
    RNAdict_lower[i.lower()]=list(set(RNAdict_lower.get(i.lower())))

In [168]:
cccc=0
final_RNA_entities_list=[]
drop_RNA_entities_list=[]
for i in RNA_entities_list:
    if RNAdict_lower.get(i.get('word').lower()):
        for j in RNAdict_lower.get(i.get('word').lower()):
            i['entity']=j
            final_RNA_entities_list.append(i)
    elif re.search(r'-[3,5]p',i.get('word')):
            if RNAdict_lower.get(re.sub(r'-[3,5]p','',i.get('word'))):
                for j in RNAdict_lower.get(re.sub(r'-[3,5]p','',i.get('word'))):
                    i['entity']=j
                    final_RNA_entities_list.append(i)
                    cccc+=1
    else:
        drop_RNA_entities_list.append(i)

In [169]:
len(final_RNA_entities_list)

1

In [170]:
len(drop_RNA_entities_list)

6

In [172]:
drop_RNA_entities_list[0]

{'word': 'miR-9',
 'wordBegin': 695649,
 'wordEnd': 695654,
 'sentenceBegin': 695594,
 'sentenceEnd': 695715,
 'sentence': 'Among all miRNAs examined, AD showed downregulation of miR-9, miR-21, miR29-b, miR-122, and miR-132 compared to controls.',
 'ner': 'HALD_MICRORNA',
 'PMID': '36771341',
 'AB': "Alzheimer's disease (AD) is the most frequent cause of dementia worldwide and represents one of the leading factors for severe disability in older persons. Although its etiology is not fully known yet, AD may develop due to multiple factors, including inflammation and oxidative stress, conditions where microRNAs (miRNAs) seem to play a pivotal role as a molecular switch. All these aspects may be modulated by nutritional factors. Among them, vitamin E has been widely studied in AD, given the plausibility of its various biological functions in influencing neurodegeneration. From a cohort of old-aged people, we measured eight vitamin E forms (tocopherols and tocotrienols), thirty cytokines/ch

In [173]:
for i in filtered_pubtator_RNAs:
    i['PMID']=i.get('id')
    i['word']=i.get('text')

In [174]:
final_RNA=filtered_pubtator_RNAs+final_RNA_entities_list

In [175]:
len(final_RNA)

52

In [176]:
RNAs_inte_copy=copy.deepcopy(final_RNA)

In [177]:
deduplicated_RNA=set()
deduplicated_RNA_count=0
RNA_list_raw=[]
for i in RNAs_inte_copy:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['JT']=rawpickwithAB.get(k).get('JT')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['target']=i.get('word')
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in deduplicated_RNA:
        deduplicated_RNA.add(tuples)
        RNA_list_raw.append(i)
    else:
        deduplicated_RNA_count+=1
        continue

In [178]:
len(RNA_list_raw)

46

In [179]:
RNA2blank={}
for i in RNA_list_raw:
    RNA2blank[i.get('entity')]=''

In [180]:
len(RNA2blank)

17

In [181]:
RNA_converter=pickle.load(open('../ncbi/RNA_converter.pkl','rb'))

In [182]:
len(RNA_converter)

339

In [183]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_RNA_info(i,final_RNAs_inte):
    ncbiid=RNA2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    # 'Also_known_as' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:',''),
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_RNAs_inte[i]=ncbisummary
    # print(ncbisummary)
    time.sleep(0.5)

In [184]:
judge=False
p = Pool(10)
manager=Manager()
final_RNAs_inte = manager.dict()
for i in RNA2blank.keys():
    if not RNA_converter.get(i):
        p.apply_async(get_RNA_info, args=(i,final_RNAs_inte))
        judge=True
p.close()
p.join()
if judge:
    RNA_converter.update(dict(final_RNAs_inte))
    pickle.dump(RNA_converter, open('../ncbi/RNA_converter.pkl','wb'))

In [185]:
len(RNA_converter)

344

In [186]:
for i in range(len(RNA_list_raw)):
    RNA_list_raw[i]=dict(RNA_list_raw[i],**RNA_converter.get(RNA_list_raw[i].get('entity')))

In [187]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in RNA_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [188]:
RNA_list_raw[0]

{'id': '36834938',
 'AB': 'Early-onset colorectal cancer (EOCRC; age younger than 50 years) incidence has been steadily increasing in recent decades worldwide. The need for new biomarkers for EOCRC prevention strategies is undeniable. In this study, we aimed to explore whether an aging factor, such as telomere length (TL), could be a useful tool in EOCRC screening. The absolute leukocyte TL from 87 microsatellite stable EOCRC patients and 109 healthy controls (HC) with the same range of age, was quantified by Real Time Quantitative PCR (RT-qPCR). Then, leukocyte whole-exome sequencing (WES) was performed to study the status of the genes involved in TL maintenance (hTERT, TERC, DKC1, TERF1, TERF2, TERF2IP, TINF2, ACD, and POT1) in 70 sporadic EOCRC cases from the original cohort. We observed that TL was significantly shorter in EOCRC patients than in healthy individuals (EOCRC mean: 122 kb vs. HC mean: 296 kb; p < 0.001), suggesting that telomeric shortening could be associated with EOC

In [189]:
for i in RNA_list_raw:
    i['addtime']=now_time

In [190]:
for i,j in RNA_converter.items():
    if j.get('Summary') and j.get('Summary')[0]=='\n':
        print(i)

collections_RNA_raw.remove({'addtime':now_time})

In [191]:
if len(RNA_list_raw)!=0:
    collections_RNA_raw.insert_many(RNA_list_raw)

# RNA_all

collections_RNA_all=db.RNA_all

In [335]:
RNA_raw_list=list(collections_RNA_raw.find())

In [196]:
for i in RNA_raw_list:
    i['type']='RNA'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [197]:
RNA_mid={}
for i in RNA_raw_list:
    if not RNA_mid.get(i.get("entity")):
        RNA_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in RNA_mid.get(i.get('entity')).get('PMID'):

            ind=RNA_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in RNA_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                RNA_mid[i.get('entity')]['sentence'][ind] = RNA_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                RNA_mid[i.get('entity')]['target'][ind] = RNA_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            RNA_mid[i.get("entity")]["PMID"]=RNA_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            RNA_mid[i.get("entity")]["sentence"]=RNA_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            RNA_mid[i.get("entity")]["AB"]=RNA_mid.get(i.get("entity")).get("AB")+i.get("AB")
            RNA_mid[i.get("entity")]["DP"]=RNA_mid.get(i.get("entity")).get("DP")+i.get("DP")
            RNA_mid[i.get("entity")]["TI"]=RNA_mid.get(i.get("entity")).get("TI")+i.get("TI")
            RNA_mid[i.get("entity")]["target"]=RNA_mid.get(i.get("entity")).get("target")+i.get("target")
            RNA_mid[i.get("entity")]["date"]=RNA_mid.get(i.get("entity")).get("date")+i.get("date")
            RNA_mid[i.get("entity")]["year"]=RNA_mid.get(i.get("entity")).get("year")+i.get("year")
            RNA_mid[i.get("entity")]["JT"]=RNA_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [198]:
RNA_output_json_final=list(RNA_mid.values())

In [199]:
len(RNA_output_json_final)

344

In [200]:
for i in RNA_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

collections_RNA_all.drop()

In [201]:
RNA_all=copy.deepcopy(RNA_output_json_final)

In [168]:
# collections_RNA_all.insert_many(RNA_all)

In [169]:
# json.dump(RNA_all_copy,open('results/RNA_all.json','w'))

# chemical_raw

In [202]:
chemical_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Chemical':
        if chemical_pubtator_addpmid.get(i.get('id')):
            chemical_pubtator_addpmid[i.get('id')]=chemical_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            chemical_pubtator_addpmid[i.get('id')]=[i]

In [203]:
print(len(chemical_pubtator_addpmid))

1257


In [204]:
chemical_output_jsons=list(chemical_pubtator_addpmid.values())

In [205]:
chemical_output_json=[]
for i in chemical_output_jsons:
    for j in i:
        chemical_output_json.append(j)

In [206]:
chemical_raw=copy.deepcopy(chemical_output_json)

In [207]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [208]:
pairs=['Toxin','Peptide','Lipid','Protein','Carbohydrate','Pharmaceutical Preparations']

In [209]:
names=locals()

In [210]:
alllist=[]
for j in pairs:
    names[j.replace(' ','_').lower()]=pickle.load(open('../chemical/'+j.replace(' ','_')+'/'+j.replace(' ','_')+'.pkl','rb'))
    alllist+=names[j.replace(' ','_').lower()][1:]

In [211]:
alllist[0]

{'text': 'Bacterial Toxins',
 'url': '/mesh/68001427',
 'MeSHUniqueID': 'D001427',
 'type': 'Toxin'}

In [212]:
for i in alllist:
    if not i.get('MeSHUniqueID'):
        i['MeSHUniqueID']='D'+i.get('url')[-6:]

In [213]:
chemicallist=[]
MESHlist=[]
notchemicallist=[]
MESHurl=[]
for i in alllist:
    if i.get('MeSHUniqueID'):
        chemicallist.append({'MESH':'MESH:'+i.get('MeSHUniqueID'),'type':i.get('type'),'url':i.get('url'),'text':i.get('text')})
        MESHlist.append('MESH:'+i.get('MeSHUniqueID'))
        MESHurl.append(i.get('url'))
    else:
        notchemicallist.append(i)

In [214]:
len(notchemicallist)

0

In [215]:
chemical2classdict={}
count=0
for i in chemicallist:
    if not chemical2classdict.get(i.get('MESH')):
        chemical2classdict[i.get('MESH')]=[i.get('type')]
    else:
        if i.get('type') in chemical2classdict.get(i.get('MESH')):
            pass
        else:
            chemical2classdict[i.get('MESH')]=chemical2classdict.get(i.get('MESH'))+[i.get('type')]
            count+=1

In [216]:
chemical2subclassdict={}
for i in chemicallist:
    chemical2subclassdict[i.get('MESH')]=i.get('text')

In [217]:
chemical2urldict={}
for i in chemicallist:
    chemical2urldict[i.get('MESH')]=i.get('url')

In [218]:
for i in chemical_raw:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [219]:
chemical_final=[]
chemical_mid=[]
chemical_dropout=[]

In [220]:
multi_chemical_type={}
for i in chemical_raw:
    if chemical2classdict.get(i.get('identifier')):
        chemical_final.append(dict(i,**{'class':chemical2classdict.get(i.get('identifier')),'subclass':chemical2subclassdict.get(i.get('identifier')),'url':chemical2urldict.get(i.get('identifier'))}))
        for j in chemical2classdict.get(i.get('identifier')):
            if len(chemical2classdict.get(i.get('identifier')))>1:
                if multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier'))):
                    if j not in multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier'))):
                        multi_chemical_type[chemical2subclassdict.get(i.get('identifier'))]=multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier')))+[j]
                else:
                    multi_chemical_type[chemical2subclassdict.get(i.get('identifier'))]=[j]
    else:
        chemical_dropout.append(i)

In [221]:
# 1012
len(chemical_final)

612

In [222]:
multi_chemical_type

{'Gangliosides': ['Lipid', 'Carbohydrate'],
 'Sulfoglycosphingolipids': ['Lipid', 'Carbohydrate'],
 'Angiotensins': ['Peptide', 'Protein'],
 'Bleomycin': ['Peptide', 'Carbohydrate'],
 'Fas Ligand Protein': ['Peptide', 'Protein'],
 'Ceramides': ['Lipid', 'Carbohydrate'],
 'Arginine Vasopressin': ['Peptide', 'Protein'],
 'Vancomycin': ['Peptide', 'Carbohydrate'],
 'Daptomycin': ['Peptide', 'Lipid'],
 'Angiotensin II': ['Peptide', 'Protein'],
 'Polymyxin B': ['Peptide', 'Lipid', 'Protein'],
 'Lipopolysaccharides': ['Toxin', 'Lipid', 'Carbohydrate']}

for i in chemical_raw:
    if i.get('identifier') in chemical2classdict.keys():
        chemical_final.append(dict(i,**{'class':chemical2classdict.get(i.get('identifier')),'subclass':chemical2subclassdict.get(i.get('identifier')),'url':chemical2urldict.get(i.get('identifier'))}))
    else:
        chemical_dropout.append(i)

In [223]:
len(chemical_dropout)

5166

In [224]:
for j in pairs:
    names[j.replace(' ','_')+'list']=[]
    for i in chemical_final:
        if j in i.get('class'):
            names[j.replace(' ','_')+'list'].append(i)
    print(len(names[j.replace(' ','_')+'list']))

1
74
288
31
254
2


In [225]:
chemical_deduplicated=set()
chemical_deduplicated_count=0
for j in pairs:
    names[j.replace(' ','_')+'list_raw']=[]
    for i in names[j.replace(' ','_')+'list']:
        k=i.get('PMID')
        i['DP']=rawpickwithAB.get(k).get('DP')
        i['AB']=rawpickwithAB.get(k).get('AB')
        i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
        i['TI']=rawpickwithAB.get(k).get('TI')
        i['entity']=i.get('subclass')
        i['target']=i.get('word')
        i['type']=j
        i['JT']=rawpickwithAB.get(k).get('JT')
        if i.get('identifier') and re.match(r'MESH:',i.get('identifier')):
            i['MESH']=i.get('identifier')[5:]
        else:
            i['MESH']=''
        # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"),i.get("type"))
        if tuples not in chemical_deduplicated:
            chemical_deduplicated.add(tuples)
            names[j.replace(' ','_')+'list_raw'].append(i)
        else:
            chemical_deduplicated_count+=1
            continue

In [226]:
chemical2ncbi={}
c=0
for j in pairs:
    for i in names[j.replace(' ','_')+'list_raw']:
        chemical2ncbi[i.get('url')]=''
        c+=1

In [227]:
# 124
len(chemical2ncbi)

100

In [228]:
chemical_converter=pickle.load(open('../ncbi/Chemical_converter.pkl','rb'))

In [229]:
len(chemical_converter)

475

In [230]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_chemical_info(i,final_chemicals_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov' + i
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    'Summary' : ''.join(html.xpath('//*[@class="mesh_ds_scope_note"]/text()')),
    'YearIntroduced' : ''.join(html.xpath('//*[@class="mesh_year_introduced"]/text()'))[17:21],
    }
    final_chemicals_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [231]:
judge=False
p = Pool(10)
manager=Manager()
final_chemicals_inte = manager.dict()
for i in chemical2ncbi.keys():
    if not chemical_converter.get(i):
        p.apply_async(get_chemical_info, args=(i,final_chemicals_inte))
        judge=True
p.close()
p.join()
if judge:
    chemical_converter.update(dict(final_chemicals_inte))
    pickle.dump(chemical_converter, open('../ncbi/Chemical_converter.pkl','wb'))

In [232]:
len(chemical_converter)

484

In [233]:
pairs

['Toxin',
 'Peptide',
 'Lipid',
 'Protein',
 'Carbohydrate',
 'Pharmaceutical Preparations']

In [234]:
for j in pairs:
    for i in range(len(names[j.replace(' ','_')+'list_raw'])):
        names[j.replace(' ','_')+'list_raw'][i]=dict(names[j.replace(' ','_')+'list_raw'][i],**chemical_converter.get(names[j.replace(' ','_')+'list_raw'][i].get('url')))

In [235]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re
for k in pairs:
    for i in names[k.replace(' ','_')+'list_raw']:
        j=i.get('DP')
        if re.search('[S,s]upplement',j,flags=re.M|re.I):
            j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
        j=re.sub('\s?-\s?','-',j)
        j=re.sub('\.','',j)
        j=re.sub('Aout','Aug',j)
        if len(j.split(' '))==3:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
            except:
                if j.split()[1] in months:
                    if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                        j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                            print('***************************************ERROR**********************************************')
                    else:
                        j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            print(j)
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif j.split()[1] not in months:
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                            middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                        else:
                            middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if middle in months:
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+middle+' '+ '1'
                        elif re.match(r'Sum(mer)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Jun'+' '+ '1'
                        elif re.match(r'Spr(ing)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Mar'+' '+ '1'
                        elif re.match(r'Win(ter)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Dec'+' '+ '1'    
                        elif re.match(r'Aut(umn)?',middle) or \
                        re.match(r'Fall',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Sep'+' '+ '1'      
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            print(j)
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                    else:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        elif len(j.split())==2:
            if re.match(r'Sum(mer)?',j.split()[1]):
                date=int(j[:4]+'0601')
            elif re.match(r'Spr(ing)?',j.split()[1]):
                date=int(j[:4]+'0301')
            elif re.match(r'Win(ter)?',j.split()[1]):
                date=int(j[:4]+'1201')
            elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
                date=int(j[:4]+'0901')
            elif re.match(r'Sum(mer)?',j.split()[0]):
                date=int(j[-4:]+'0601')
            elif re.match(r'Spr(ing)?',j.split()[0]):
                date=int(j[-4:]+'0301')
            elif re.match(r'Win(ter)?',j.split()[0]):
                date=int(j[-4:]+'1201')
            elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
                date=int(j[-4:]+'0901')
            else:
                try:
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
                except:
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                            middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                        else:
                            middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if middle in months:
                            j=j.split()[0]+' '+middle+' '+ '1'
                        elif re.match(r'Sum(mer)?',middle):
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                        elif re.match(r'Spr(ing)?',middle):
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                        elif re.match(r'Win(ter)?',middle):
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                        elif re.match(r'Aut(umn)?',middle) or \
                        re.match(r'Fall',middle):
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                    elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if len(midd)!=2:
                            midd='0'+midd
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                    else:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        elif len(j.split())==1:
            j=j[:4]
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        else:
            j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        i['date']=date
        i['year']=int(str(i.get('date'))[:4])

In [236]:
for k in pairs:
    names[k.replace(' ','_')+'_raw']=copy.deepcopy(names[k.replace(' ','_')+'list_raw'])
    for i in names[k.replace(' ','_')+'_raw']:
        i['addtime']=now_time

# Carbohydrate_raw

In [237]:
collections_Carbohydrate_raw=db.Carbohydrate_raw

In [238]:
len(Carbohydrate_raw)

239

In [239]:
Carbohydrate_raw[0]

{'AB': "Systemic immunity supports lifelong brain function. Obesity posits a chronic burden on systemic immunity. Independently, obesity was shown as a risk factor for Alzheimer's disease (AD). Here we show that high-fat obesogenic diet accelerated recognition-memory impairment in an AD mouse model (5xFAD). In obese 5xFAD mice, hippocampal cells displayed only minor diet-related transcriptional changes, whereas the splenic immune landscape exhibited aging-like CD4+ T-cell deregulation. Following plasma metabolite profiling, we identified free N-acetylneuraminic acid (NANA), the predominant sialic acid, as the metabolite linking recognition-memory impairment to increased splenic immune-suppressive cells in mice. Single-nucleus RNA-sequencing revealed mouse visceral adipose macrophages as a potential source of NANA. In vitro, NANA reduced CD4+ T-cell proliferation, tested in both mouse and human. In vivo, NANA administration to standard diet-fed mice recapitulated high-fat diet effects o

In [240]:
if len(Carbohydrate_raw)!=0:
    collections_Carbohydrate_raw.insert_many(Carbohydrate_raw)

In [241]:
shenme=list(collections_Carbohydrate_raw.find())

In [243]:
shenme[-1]

{'_id': ObjectId('640ff7b7e6762ddb334129a5'),
 'AB': ' As a follow-up to an earlier study demonstrating healthier cardiometabolic profiles among long-term multiple dietary supplement (LTMDS) users, we examined if cardiometabolic health benefits associated with LTMDS use persisted with aging. The study is based on LTMDS users from North America and 2007-2010 NHANES participants who were used for comparison to the LTMDS users. NHANES subjects were classified as non-dietary supplement (NS) users, single supplement/single purpose supplement (SS) users, multivitamin/mineral supplement (MVMS) users, and multiple dietary supplement (MDS) users. Supplement groups were compared for total, HDL and LDL cholesterol; triglycerides; glucose; insulin; CRP and HbA1c >= 6.5%, adjusting for age, sex, income, education, BMI, history of CVD, and medications for hypercholesterolemia and diabetes. Geometric mean (95% confidence interval) LDL cholesterol was significantly lower (P < 0.05) for all supplement 

# Carbohydrate_all

collections_Carbohydrate_all=db.Carbohydrate_all

collections_Carbohydrate_all.remove({'addtime':now_time})

In [244]:
Carbohydrate_raw_all=list(collections_Carbohydrate_raw.find())

In [245]:
len(Carbohydrate_raw_all)

17629

In [246]:
for i in Carbohydrate_raw_all:
    i['type']='Carbohydrate'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [247]:
Carbohydrate_mid={}
for i in Carbohydrate_raw_all:
    if not Carbohydrate_mid.get(i.get("entity")):
        Carbohydrate_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Carbohydrate_mid.get(i.get('entity')).get('PMID'):

            ind=Carbohydrate_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Carbohydrate_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Carbohydrate_mid[i.get('entity')]['sentence'][ind] = Carbohydrate_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Carbohydrate_mid[i.get('entity')]['target'][ind] = Carbohydrate_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Carbohydrate_mid[i.get("entity")]["PMID"]=Carbohydrate_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Carbohydrate_mid[i.get("entity")]["sentence"]=Carbohydrate_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Carbohydrate_mid[i.get("entity")]["AB"]=Carbohydrate_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Carbohydrate_mid[i.get("entity")]["DP"]=Carbohydrate_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Carbohydrate_mid[i.get("entity")]["TI"]=Carbohydrate_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Carbohydrate_mid[i.get("entity")]["target"]=Carbohydrate_mid.get(i.get("entity")).get("target")+i.get("target")
            Carbohydrate_mid[i.get("entity")]["date"]=Carbohydrate_mid.get(i.get("entity")).get("date")+i.get("date")
            Carbohydrate_mid[i.get("entity")]["year"]=Carbohydrate_mid.get(i.get("entity")).get("year")+i.get("year")
            Carbohydrate_mid[i.get("entity")]["JT"]=Carbohydrate_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [248]:
Carbohydrate_all=list(Carbohydrate_mid.values())

In [249]:
len(Carbohydrate_all)

203

In [250]:
for i in Carbohydrate_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [251]:
Carbohydrate_all[10].get('Summary')

'A class of sphingolipids found largely in the brain and other nervous tissue. They contain phosphocholine or phosphoethanolamine as their polar head group so therefore are the only sphingolipids classified as PHOSPHOLIPIDS.'

In [789]:
# collections_Carbohydrate_all.insert_many(Carbohydrate_all)

# Lipid_raw

In [252]:
collections_Lipid_raw=db.Lipid_raw

In [253]:
len(Lipid_raw)

273

In [254]:
if len(Lipid_raw)!=0:
    collections_Lipid_raw.insert_many(Lipid_raw)

# Lipid_all

collections_Lipid_all=db.Lipid_all

collections_Lipid_all.remove({'addtime':now_time})

In [255]:
Lipid_raw_all=list(collections_Lipid_raw.find())

In [256]:
len(Lipid_raw_all)

19132

In [257]:
for i in Lipid_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)

In [258]:
for i in Lipid_raw_all:
    i['type']='Lipid'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [259]:
Lipid_mid={}
for i in Lipid_raw_all:
    if not Lipid_mid.get(i.get("entity")):
        Lipid_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Lipid_mid.get(i.get('entity')).get('PMID'):

            ind=Lipid_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Lipid_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Lipid_mid[i.get('entity')]['sentence'][ind] = Lipid_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Lipid_mid[i.get('entity')]['target'][ind] = Lipid_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Lipid_mid[i.get("entity")]["PMID"]=Lipid_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Lipid_mid[i.get("entity")]["sentence"]=Lipid_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Lipid_mid[i.get("entity")]["AB"]=Lipid_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Lipid_mid[i.get("entity")]["DP"]=Lipid_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Lipid_mid[i.get("entity")]["TI"]=Lipid_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Lipid_mid[i.get("entity")]["target"]=Lipid_mid.get(i.get("entity")).get("target")+i.get("target")
            Lipid_mid[i.get("entity")]["date"]=Lipid_mid.get(i.get("entity")).get("date")+i.get("date")
            Lipid_mid[i.get("entity")]["year"]=Lipid_mid.get(i.get("entity")).get("year")+i.get("year")
            Lipid_mid[i.get("entity")]["JT"]=Lipid_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [260]:
Lipid_all=list(Lipid_mid.values())

In [261]:
len(Lipid_all)

191

In [262]:
for i in Lipid_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [803]:
# collections_Lipid_all.insert_many(Lipid_all)

# Peptide_raw

In [263]:
collections_Peptide_raw=db.Peptide_raw

In [264]:
len(Peptide_raw)

67

In [265]:
if len(Peptide_raw)!=0:
    collections_Peptide_raw.insert_many(Peptide_raw)

# Peptide_all

In [266]:
collections_Peptide_all=db.Peptide_all

collections_Peptide_all.remove({'addtime':now_time})

In [267]:
Peptide_raw_all=list(collections_Peptide_raw.find())

In [268]:
len(Peptide_raw_all)

3412

In [269]:
for i in Peptide_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)
        break

{'_id': ObjectId('64005023648b16758fce3875'), 'AB': 'AIMS: Real-world evidence about heart failure (HF) medical care for elderly patients is limited. The aim of this study was to clarify the age-dependent associations of HF medications at discharge with clinical outcomes in a super-aged society using a real-world big database. METHODS AND RESULTS: Patients hospitalized with a first episode of acute HF in 2013-2019 were identified in a nationwide claims database in Japan and were analyzed based on the HF medications at discharge. Hazard ratios (HRs) and 95 % confidence intervals (95 % CIs) for clinical outcomes were calculated after adjustment. Among 325,468 patients from 4351 hospitals, 130,230 (40.0 %) were aged >= 85 years. All-cause mortality and HF readmission rate within 1 year were 14 % and 23 %, respectively. beta-Blockers and angiotensin-converting-enzyme inhibitors or angiotensin II receptor blockers (ACEI/ARB) uses were associated with lower mortality (HR [95 %CI]: 0.84 [0.83

In [270]:
for i in Peptide_raw_all:
    i['type']='Peptide'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [271]:
Peptide_mid={}
for i in Peptide_raw_all:
    if not Peptide_mid.get(i.get("entity")):
        Peptide_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Peptide_mid.get(i.get('entity')).get('PMID'):

            ind=Peptide_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Peptide_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Peptide_mid[i.get('entity')]['sentence'][ind] = Peptide_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Peptide_mid[i.get('entity')]['target'][ind] = Peptide_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Peptide_mid[i.get("entity")]["PMID"]=Peptide_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Peptide_mid[i.get("entity")]["sentence"]=Peptide_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Peptide_mid[i.get("entity")]["AB"]=Peptide_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Peptide_mid[i.get("entity")]["DP"]=Peptide_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Peptide_mid[i.get("entity")]["TI"]=Peptide_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Peptide_mid[i.get("entity")]["target"]=Peptide_mid.get(i.get("entity")).get("target")+i.get("target")
            Peptide_mid[i.get("entity")]["date"]=Peptide_mid.get(i.get("entity")).get("date")+i.get("date")
            Peptide_mid[i.get("entity")]["year"]=Peptide_mid.get(i.get("entity")).get("year")+i.get("year")
            Peptide_mid[i.get("entity")]["JT"]=Peptide_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [272]:
Peptide_all=list(Peptide_mid.values())

In [273]:
len(Peptide_all)

79

In [274]:
for i in Peptide_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [815]:
# collections_Peptide_all.insert_many(Peptide_all)

# Pharmaceutical_Preparations_raw

In [275]:
collections_Pharmaceutical_Preparations_raw=db.Pharmaceutical_Preparations_raw

In [276]:
len(Pharmaceutical_Preparations_raw)

2

In [277]:
if len(Pharmaceutical_Preparations_raw)!=0:
    collections_Pharmaceutical_Preparations_raw.insert_many(Pharmaceutical_Preparations_raw)

# Pharmaceutical_Preparations_all

In [278]:
collections_Pharmaceutical_Preparations_all=db.Pharmaceutical_Preparations_all

collections_Pharmaceutical_Preparations_all.remove({'addtime':now_time})

In [279]:
Pharmaceutical_Preparations_raw_all=list(collections_Pharmaceutical_Preparations_raw.find())

In [280]:
len(Pharmaceutical_Preparations_raw_all)

269

In [281]:
for i in Pharmaceutical_Preparations_raw_all:
    i['type']='Pharmaceutical Preparations'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [282]:
Pharmaceutical_Preparations_mid={}
for i in Pharmaceutical_Preparations_raw_all:
    if not Pharmaceutical_Preparations_mid.get(i.get("entity")):
        Pharmaceutical_Preparations_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Pharmaceutical_Preparations_mid.get(i.get('entity')).get('PMID'):

            ind=Pharmaceutical_Preparations_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Pharmaceutical_Preparations_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Pharmaceutical_Preparations_mid[i.get('entity')]['sentence'][ind] = Pharmaceutical_Preparations_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Pharmaceutical_Preparations_mid[i.get('entity')]['target'][ind] = Pharmaceutical_Preparations_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Pharmaceutical_Preparations_mid[i.get("entity")]["PMID"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Pharmaceutical_Preparations_mid[i.get("entity")]["sentence"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Pharmaceutical_Preparations_mid[i.get("entity")]["AB"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Pharmaceutical_Preparations_mid[i.get("entity")]["DP"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Pharmaceutical_Preparations_mid[i.get("entity")]["TI"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Pharmaceutical_Preparations_mid[i.get("entity")]["target"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("target")+i.get("target")
            Pharmaceutical_Preparations_mid[i.get("entity")]["date"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("date")+i.get("date")
            Pharmaceutical_Preparations_mid[i.get("entity")]["year"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("year")+i.get("year")
            Pharmaceutical_Preparations_mid[i.get("entity")]["JT"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [283]:
Pharmaceutical_Preparations_all=list(Pharmaceutical_Preparations_mid.values())

In [284]:
len(Pharmaceutical_Preparations_all)

11

In [285]:
for i in Pharmaceutical_Preparations_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [286]:
# collections_Pharmaceutical_Preparations_all.insert_many(Pharmaceutical_Preparations_all)

# Protein_raw

In [287]:
collections_Protein_raw=db.Protein_raw

In [288]:
len(Protein_raw)

29

In [289]:
if len(Protein_raw)!=0:
    collections_Protein_raw.insert_many(Protein_raw)

# Protein_all

In [290]:
collections_Protein_all=db.Protein_all

collections_Protein_all.remove({'addtime':now_time})

In [337]:
Protein_raw_all=list(collections_Protein_raw.find())

In [292]:
len(Protein_raw_all)

1857

In [293]:
for i in Protein_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)


{'_id': ObjectId('6400506e648b16758fce4078'), 'AB': 'AIMS: Real-world evidence about heart failure (HF) medical care for elderly patients is limited. The aim of this study was to clarify the age-dependent associations of HF medications at discharge with clinical outcomes in a super-aged society using a real-world big database. METHODS AND RESULTS: Patients hospitalized with a first episode of acute HF in 2013-2019 were identified in a nationwide claims database in Japan and were analyzed based on the HF medications at discharge. Hazard ratios (HRs) and 95 % confidence intervals (95 % CIs) for clinical outcomes were calculated after adjustment. Among 325,468 patients from 4351 hospitals, 130,230 (40.0 %) were aged >= 85 years. All-cause mortality and HF readmission rate within 1 year were 14 % and 23 %, respectively. beta-Blockers and angiotensin-converting-enzyme inhibitors or angiotensin II receptor blockers (ACEI/ARB) uses were associated with lower mortality (HR [95 %CI]: 0.84 [0.83

In [294]:
for i in Protein_raw_all:
    i['type']='Protein'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [295]:
Protein_mid={}
for i in Protein_raw_all:
    if not Protein_mid.get(i.get("entity")):
        Protein_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Protein_mid.get(i.get('entity')).get('PMID'):

            ind=Protein_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Protein_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Protein_mid[i.get('entity')]['sentence'][ind] = Protein_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Protein_mid[i.get('entity')]['target'][ind] = Protein_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Protein_mid[i.get("entity")]["PMID"]=Protein_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Protein_mid[i.get("entity")]["sentence"]=Protein_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Protein_mid[i.get("entity")]["AB"]=Protein_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Protein_mid[i.get("entity")]["DP"]=Protein_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Protein_mid[i.get("entity")]["TI"]=Protein_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Protein_mid[i.get("entity")]["target"]=Protein_mid.get(i.get("entity")).get("target")+i.get("target")
            Protein_mid[i.get("entity")]["date"]=Protein_mid.get(i.get("entity")).get("date")+i.get("date")
            Protein_mid[i.get("entity")]["year"]=Protein_mid.get(i.get("entity")).get("year")+i.get("year")
            Protein_mid[i.get("entity")]["JT"]=Protein_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [296]:
Protein_all=list(Protein_mid.values())

In [297]:
len(Protein_all)

29

In [298]:
for i in Protein_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [299]:
# collections_Protein_all.insert_many(Protein_all)

# Toxin_raw

In [300]:
collections_Toxin_raw=db.Toxin_raw

In [346]:
len(Toxin_raw)

1

In [347]:
if len(Toxin_raw)!=0:
    collections_Toxin_raw.insert_many(Toxin_raw)

# Toxin_all

In [348]:
collections_Toxin_all=db.Toxin_all

collections_Toxin_all.remove({'addtime':now_time})

In [349]:
Toxin_raw_all=list(collections_Toxin_raw.find())

toxin_bk=copy.deepcopy(Toxin_raw_all)

In [350]:
len(Toxin_raw_all)

540

In [351]:
for i in Toxin_raw_all:
    i['type']='Toxin'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [352]:
Toxin_mid={}
for i in Toxin_raw_all:
    if not Toxin_mid.get(i.get("entity")):
        Toxin_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Toxin_mid.get(i.get('entity')).get('PMID'):

            ind=Toxin_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Toxin_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Toxin_mid[i.get('entity')]['sentence'][ind] = Toxin_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Toxin_mid[i.get('entity')]['target'][ind] = Toxin_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Toxin_mid[i.get("entity")]["PMID"]=Toxin_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Toxin_mid[i.get("entity")]["sentence"]=Toxin_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Toxin_mid[i.get("entity")]["AB"]=Toxin_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Toxin_mid[i.get("entity")]["DP"]=Toxin_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Toxin_mid[i.get("entity")]["TI"]=Toxin_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Toxin_mid[i.get("entity")]["target"]=Toxin_mid.get(i.get("entity")).get("target")+i.get("target")
            Toxin_mid[i.get("entity")]["date"]=Toxin_mid.get(i.get("entity")).get("date")+i.get("date")
            Toxin_mid[i.get("entity")]["year"]=Toxin_mid.get(i.get("entity")).get("year")+i.get("year")
            Toxin_mid[i.get("entity")]["JT"]=Toxin_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [353]:
Toxin_all=list(Toxin_mid.values())

In [354]:
len(Toxin_all)

18

In [355]:
for i in Toxin_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [851]:
# collections_Toxin_all.insert_many(Toxin_all)

# Mutation_raw

In [356]:
collections_Mutation_raw=db.Mutation_raw

collections_Mutation_raw.remove({'addtime':now_time})

In [357]:
Mutation_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Mutation':
        if Mutation_pubtator_addpmid.get(i.get('id')):
            Mutation_pubtator_addpmid[i.get('id')]=Mutation_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            Mutation_pubtator_addpmid[i.get('id')]=[i]

In [358]:
Mutation_output_json=list(Mutation_pubtator_addpmid.values())

In [359]:
Mutation_raw=sum(Mutation_output_json,[])

In [360]:
len(Mutation_raw)

141

In [361]:
for i in Mutation_raw:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [362]:
Mutation_deduplicated=set()
Mutation_deduplicated_count=0
mutation_list_raw=[]
for i in Mutation_raw:
    if i.get('identifier'):
        k=i.get('PMID')
        i['DP']=rawpickwithAB.get(k).get('DP')
        i['AB']=rawpickwithAB.get(k).get('AB')
        i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
        i['TI']=rawpickwithAB.get(k).get('TI')
        i['entity']=i.get('identifier')
        i['target']=i.get('word')
        i['type']='Mutation'
        i['JT']=rawpickwithAB.get(k).get('JT')
        # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        if tuples not in Mutation_deduplicated:
            Mutation_deduplicated.add(tuples)
            mutation_list_raw.append(i)
        else:
            Mutation_deduplicated_count+=1
            continue

In [363]:
len(mutation_list_raw)

129

In [364]:
import re
dropout1=[]
dropout2=[]
for i in mutation_list_raw:
    if re.match(r'rs\d+',i.get('identifier')):
        dropout1.append(i)
    else:
        dropout2.append(i)

In [365]:
len(dropout1)

106

In [366]:
len(dropout2)

23

In [367]:
mutation2ncbi={}
for i in dropout1:
    mutation2ncbi[i.get('entity')]={}

In [368]:
len(mutation2ncbi)

65

In [369]:
mutation_converter=pickle.load(open('../ncbi/Mutation_converter.pkl','rb'))

In [370]:
len(mutation_converter)

1853

In [371]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_mutation_info(i,final_mutations_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/snp/' + i
    html = etree.HTML(requests.get(url=url,headers=headers).content)
    ncbisummary={
    'Position':''.join(html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[1]/text()'))+''.join(html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[2]/text()')),
    # 'Position':html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[1]/text()')+html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[2]/text()'),
    'Alleles':''.join(html.xpath('normalize-space(//*[@class="usa-width-one-half"][1]/dd[3]/text())')),
    'Clinical Significance':'https://www.ncbi.nlm.nih.gov'+''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[1]/a/@href')) if html.xpath('//*[@class="usa-width-one-half"][2]/dd[1]/a/@href') else '',
    'Gene : Consequence':[''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/div[1]/text()')),''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/span[1]/text()'))] if ''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/div[1]/text()')) else [''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/span[1]/text()'))],
    'Genomic View':url+''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[4]/a/@href')),
    }
    final_mutations_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [372]:
judge=False
p = Pool(10)
manager=Manager()
final_mutations_inte = manager.dict()
for i in mutation2ncbi.keys():
    if not mutation_converter.get(i):
        p.apply_async(get_mutation_info, args=(i,final_mutations_inte))
        judge=True
p.close()
p.join()
if judge:
    mutation_converter.update(dict(final_mutations_inte))
    pickle.dump(mutation_converter, open('../ncbi/Mutation_converter.pkl','wb'))

In [373]:
len(mutation_converter)

1893

In [374]:
for i in range(len(mutation_list_raw)):
    if re.match(r'rs\d+',mutation_list_raw[i].get('entity')):
        mutation_list_raw[i]=dict(mutation_list_raw[i],**mutation_converter.get(mutation_list_raw[i].get('entity')))
    else:
        mutation_list_raw[i]=dict(mutation_list_raw[i],**{'Position':'','Alleles':'','Clinical Significance':'','Gene : Consequence':[''],'Genomic View':''})

In [375]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in mutation_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [376]:
len(mutation_list_raw)

129

In [377]:
mutation_list_raw[0]

{'AB': 'Osteoarthritis (OA) pathogenesis is associated with reduced chondrocyte homeostasis and increased levels of cartilage cellular senescence. Chondrosenescence is the development of cartilage senescence that increases with aging joints and disrupts chondrocyte homeostasis and is associated with OA. Adenosine A2A receptor (A2AR) activation in cartilage via intra-articular injection of liposomal A2AR agonist, liposomal-CGS21680, leads to cartilage regeneration in vivo and chondrocyte homeostasis. A2AR knockout mice develop early OA isolated chondrocytes demonstrate upregulated expression of cellular senescence and aging-associated genes. Based on these observations, we hypothesized that A2AR activation would ameliorate cartilage senescence. We found that A2AR stimulation of chondrocytes reduced beta-galactosidase staining and regulated levels and cell localization of common senescence mediators p21 and p16 in vitro in the human TC28a2 chondrocyte cell line. In vivo analysis similarl

In [378]:
Mutation_raw=copy.deepcopy(mutation_list_raw)

In [379]:
collections_Mutation_raw.insert_many(Mutation_raw)

# Mutation_all

collections_Mutation_all=db.Mutation_all

In [380]:
Mutation_testcopy=list(collections_Mutation_raw.find())

In [381]:
len(Mutation_testcopy)

6881

In [382]:
for i in Mutation_testcopy:
    i['type']='Mutation'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [383]:
Mutation_mid={}
for i in Mutation_testcopy:
    if not Mutation_mid.get(i.get("entity")):
        Mutation_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Mutation_mid.get(i.get('entity')).get('PMID'):

            ind=Mutation_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Mutation_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Mutation_mid[i.get('entity')]['sentence'][ind] = Mutation_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Mutation_mid[i.get('entity')]['target'][ind] = Mutation_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Mutation_mid[i.get("entity")]["PMID"]=Mutation_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Mutation_mid[i.get("entity")]["sentence"]=Mutation_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Mutation_mid[i.get("entity")]["AB"]=Mutation_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Mutation_mid[i.get("entity")]["DP"]=Mutation_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Mutation_mid[i.get("entity")]["TI"]=Mutation_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Mutation_mid[i.get("entity")]["target"]=Mutation_mid.get(i.get("entity")).get("target")+i.get("target")
            Mutation_mid[i.get("entity")]["date"]=Mutation_mid.get(i.get("entity")).get("date")+i.get("date")
            Mutation_mid[i.get("entity")]["year"]=Mutation_mid.get(i.get("entity")).get("year")+i.get("year")
            Mutation_mid[i.get("entity")]["JT"]=Mutation_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [384]:
Mutation_all=list(Mutation_mid.values())

In [385]:
for i in Mutation_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [386]:
len(Mutation_all)

2668

# Disease_raw

In [387]:
collections_Disease_raw=db.Disease_raw

collections_Disease_raw.remove({'addtime':now_time})

In [388]:
Disease_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Disease':
        if Disease_pubtator_addpmid.get(i.get('id')):
            Disease_pubtator_addpmid[i.get('id')]=Disease_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            Disease_pubtator_addpmid[i.get('id')]=[i]

In [389]:
Disease_output_json=[]
for i,j in Disease_pubtator_addpmid.items():
    Disease_output_json+=j

In [390]:
len(Disease_output_json)

22317

In [391]:
Disease_mesh=pickle.load(open('../disease/Disease.pkl','rb'))

In [392]:
Disease_mesh=Disease_mesh[1:]

In [393]:
for i in Disease_mesh:
    if i.get('text')=='Uveitis, Suppurative':
        i['MeSHUniqueID']='D015829'

In [394]:
for i in Disease_mesh:
    if i.get('text')=='Hearing Loss':
        print(i)

{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}
{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}
{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}


In [395]:
diseaselist=[]
notdiseaselist=[]
for i in Disease_mesh:
    if i.get('MeSHUniqueID'):
        diseaselist.append({'MESH':'MESH:'+i.get('MeSHUniqueID'),'type':i.get('type'),'url':i.get('url'),'text':i.get('text')})
    else:
        notdiseaselist.append(i)

In [396]:
len(diseaselist)

12791

In [397]:
len(notdiseaselist)

0

In [398]:
disease2classdict={}
count=0
for i in diseaselist:
    if not disease2classdict.get(i.get('MESH')):
        disease2classdict[i.get('MESH')]=[i.get('type')]
    else:
        if i.get('type') in disease2classdict.get(i.get('MESH')):
            pass
        else:
            disease2classdict[i.get('MESH')]=disease2classdict.get(i.get('MESH'))+[i.get('type')]
            count+=1
            if count<=5:
                print(i.get('MESH')+'\t'+str(disease2classdict.get(i.get('MESH'))))

In [399]:
disease2subclassdict={}
count=0
for i in diseaselist:
    disease2subclassdict[i.get('MESH')]=i.get('text')

In [400]:
disease2urldict={}
count=0
for i in diseaselist:
    disease2urldict[i.get('MESH')]=i.get('url')

In [401]:
len(disease2classdict)

4915

In [402]:
for i in Disease_output_json:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [403]:
Disease_final=[]
Disease_dropout=[]

In [404]:
for i in Disease_output_json:
    if i.get('identifier') in disease2classdict.keys():
        Disease_final.append(dict(i,**{'class':disease2classdict.get(i.get('identifier')),'subclass':disease2subclassdict.get(i.get('identifier')),'url':disease2urldict.get(i.get('identifier'))}))
    else:
        Disease_dropout.append(i)

In [405]:
len(Disease_output_json)

22317

In [406]:
len(Disease_dropout)

4924

In [407]:
Disease_deduplicated=set()
Disease_deduplicated_count=0
disease_list_raw=[]
for i in Disease_final:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['entity']=i.get('subclass')
    i['target']=i.get('word')
    i['type']='Disease'
    i['JT']=rawpickwithAB.get(k).get('JT')
    if i.get('identifier') and re.match(r'MESH:',i.get('identifier')):
        i['MESH']=i.get('identifier')[5:]
    else:
        i['MESH']=''
    # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Disease_deduplicated:
        Disease_deduplicated.add(tuples)
        disease_list_raw.append(i)
    else:
        Disease_deduplicated_count+=1
        continue

In [408]:
Disease_deduplicated_count

1070

In [409]:
disease2ncbi={}
for i in disease_list_raw:
    disease2ncbi[i.get('url')]=''

In [410]:
len(disease2ncbi)

998

In [411]:
disease_converter=pickle.load(open('../ncbi/Disease_converter.pkl','rb'))

In [412]:
len(disease_converter)

3344

In [413]:
import time
import datetime
from multiprocessing import Process, Manager, Pool
import os, time, random
from lxml import etree
import requests
def get_disease_info(i,final_diseases_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov' + i
    
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Summary' : ''.join(html.xpath('//*[@class="mesh_ds_scope_note"]/text()'))
    }
    final_diseases_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [414]:
judge=False
p = Pool(10)
manager=Manager()
final_diseases_inte = manager.dict()
for i in disease2ncbi.keys():
    if not disease_converter.get(i):
        p.apply_async(get_disease_info, args=(i,final_diseases_inte))
        judge=True
p.close()
p.join()
if judge:
    disease_converter.update(dict(final_diseases_inte))
    pickle.dump(disease_converter, open('../ncbi/Disease_converter.pkl','wb'))

In [415]:
len(disease_converter)

3347

In [416]:
for i in range(len(disease_list_raw)):
    disease_list_raw[i]=dict(disease_list_raw[i],**disease_converter.get(disease_list_raw[i].get('url')))

In [417]:
Disease_deduplicated_dropout=set()
Disease_deduplicated_dropout_count=0
Disease_dropout_raw=[]
for i in Disease_dropout:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['entity']=i.get('word')
    i['target']=i.get('word')
    i['type']='Disease'
    i['JT']=rawpickwithAB.get(k).get('JT')
    if i.get('identifier') and re.match('MESH:',i.get('identifier')):
        i['MESH']=i.get('identifier')[5:]
    else:
        i['MESH']=''
    # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Disease_deduplicated_dropout:
        Disease_deduplicated_dropout.add(tuples)
        Disease_dropout_raw.append(i)
    else:
        Disease_deduplicated_dropout_count+=1
        continue

In [418]:
for i in Disease_dropout_raw:
    i['Summary']=''

In [419]:
len(Disease_dropout_raw)

4647

In [420]:
Disease_raw=disease_list_raw+Disease_dropout_raw

In [421]:
len(Disease_raw)

20970

In [422]:
import time
import datetime
import re
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
import re

for i in Disease_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [423]:
collections_Disease_raw.insert_many(Disease_raw)

# Disease_all

collections_Disease_all=db.Disease_all

In [464]:
Disease_raw_list=list(collections_Disease_raw.find())

In [473]:
for i in Disease_raw_list:
    if i.get('word').upper()=="ALZHEI-MER'S DISEASE":
        i['entity']='Alzheimer Disease'

for i in disease_all_today:
    if i.get('Summary') and not i.get('Mesh'):
        print(i.get('Summary'))
        print(i.get('entity'))
        print(i.get('addtime'))
        print(i.get('identifier'))
        print(i.get('Mesh'))
        break

In [ ]:
for i in Disease_raw_list:
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [ ]:
i

In [ ]:
Disease_mid={}
for i in Disease_raw_list:
    if not Disease_mid.get(i.get("entity")):
        Disease_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Disease_mid.get(i.get('entity')).get('PMID'):

            ind=Disease_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Disease_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Disease_mid[i.get('entity')]['sentence'][ind] = Disease_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Disease_mid[i.get('entity')]['target'][ind] = Disease_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Disease_mid[i.get("entity")]["PMID"]=Disease_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Disease_mid[i.get("entity")]["sentence"]=Disease_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Disease_mid[i.get("entity")]["AB"]=Disease_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Disease_mid[i.get("entity")]["DP"]=Disease_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Disease_mid[i.get("entity")]["TI"]=Disease_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Disease_mid[i.get("entity")]["target"]=Disease_mid.get(i.get("entity")).get("target")+i.get("target")
            Disease_mid[i.get("entity")]["date"]=Disease_mid.get(i.get("entity")).get("date")+i.get("date")
            Disease_mid[i.get("entity")]["year"]=Disease_mid.get(i.get("entity")).get("year")+i.get("year")
            Disease_mid[i.get("entity")]["JT"]=Disease_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [ ]:
Disease_output_json_final=list(Disease_mid.values())

In [ ]:
# 17330
len(Disease_output_json_final)

In [ ]:
for i in Disease_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [ ]:
Disease_all=copy.deepcopy(Disease_output_json_final)

In [ ]:
len(Disease_all)

In [512]:
pwd

'/root/hald_autoupdate/2023_03_13'

In [ ]:
pwd

In [17]:
names=locals()

In [18]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

for i in entity_pairs:
    for j in names[i+'_all']:
        j['addtime']=now_time

In [19]:
for i in entity_pairs:
    pickle.dump(names[i+'_all'],open('results/step2/'+i+'_all.pkl','wb'))

KeyError: 'Gene_all'

In [20]:
from nltk.tokenize import sent_tokenize

In [21]:
for i in entity_pairs:
    names[i+'_all']=pickle.load(open('results/step2/'+i+'_all.pkl','rb'))

In [24]:
Gene_all[10].get('NCBI_URL')

'https://www.ncbi.nlm.nih.gov/gene/1113'

In [26]:
Gene_all[10].get('NCBI_ID')

'1113'

In [25]:
ppp=0
for i in Gene_all:
    if not i.get('NCBI_URL'):
        ppp+=1
ppp

0

In [28]:
ppp=0
for i in RNA_all:
    if not i.get('NCBI_URL'):
        ppp+=1
ppp

0

In [23]:
Gene_all[10].keys()

dict_keys(['_id', 'AB', 'type', 'identifier', 'sentence', 'sentenceSize', 'sentenceIndex', 'entity', 'ner', 'PMID', 'word', 'jnlpba_tags', 'bc5cdr_tags', 'bionlp13cg_tags', 'ScispaCy', 'bern', 'DP', 'URL', 'TI', 'target', 'JT', 'Official_Symbol', 'Official_Full_Name', 'HGNC_ID', 'HGNC_URL', 'Ensembl_ID', 'Ensembl_URL', 'MIM_ID', 'MIM_URL', 'AllianceGenome_ID', 'AllianceGenome_URL', 'Gene_type', 'Lineage', 'Also_known_as', 'Summary', 'NCBI_ID', 'NCBI_URL', 'date', 'year', 'addtime', 'allyears', 'datestring', 'targetshapes'])

In [27]:
RNA_all[10].keys()

dict_keys(['_id', 'AB', 'type', 'identifier', 'sentence', 'sentenceSize', 'sentenceIndex', 'entity', 'ner', 'specific_type', 'PMID', 'word', 'DP', 'URL', 'TI', 'target', 'Official_Symbol', 'Official_Full_Name', 'HGNC_ID', 'HGNC_URL', 'Ensembl_ID', 'Ensembl_URL', 'MIM_ID', 'MIM_URL', 'AllianceGenome_ID', 'AllianceGenome_URL', 'Gene_type', 'Lineage', 'Also_known_as', 'Summary', 'Expression', 'NCBI_ID', 'NCBI_URL', 'date', 'year', 'JT', 'addtime', 'allyears', 'datestring', 'targetshapes'])

In [29]:
all_es=[]
mu=0
di=0
noturl=[]
for i in Gene_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Also_known_as':i.get('Also_known_as'),'Summary':sent_tokenize(i.get('Summary'))[0],'NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Also_known_as':i.get('Also_known_as'),'Summary':'','NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in RNA_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'specific_type':i.get('specific_type'),'Also_known_as':i.get('Also_known_as'),'Summary':sent_tokenize(i.get('Summary'))[0],'NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'specific_type':i.get('specific_type'),'Also_known_as':i.get('Also_known_as'),'Summary':'','NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Carbohydrate_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Lipid_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Peptide_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Pharmaceutical_Preparations_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':'PharmaceuticalPreparations'+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':'PharmaceuticalPreparations'+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Protein_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Toxin_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Mutation_all:
    if i.get('Genomic View'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Genomic_View':i.get('Genomic View'),'Alleles':i.get('Alleles'),'Position':i.get('Position'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        mu+=1
for i in Disease_all:
    if i.get('url'):
        if i.get('Summary'):
            all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'Summary':sent_tokenize(i.get('Summary'))[0],'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
        else:
            all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'Summary':'','_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        di+=1
        noturl.append(i)

In [30]:
all_es_copy=copy.deepcopy(all_es)

In [31]:
#10907
len(all_es_copy)

11068

In [32]:
for i in all_es:
    i['allwordsdict']={}
    if i.get('Also_known_as'):
        i['allwords']=i.get('allwords',[])+i.get('Also_known_as').split('; ')
        for l in i.get('Also_known_as').split('; '):
            i['allwordsdict'][l.upper()]=l
    for j in i.get('targetshapes'):
        if j.upper() != i.get('entity').upper() and not i.get('allwordsdict').get(j.upper()):
            i['allwords']=i.get('allwords',[])+[j.upper()]
            i['allwordsdict'][j.upper()]=j
    if i.get('targetshapes'):
        del i['targetshapes']
    del i['allwordsdict']

In [33]:
all_es[-1000]

{'entity': 'Purpura, Thrombocytopenic',
 'PMID': ['30593149', '30010502', '26316484', '17124090', '1566747'],
 'literature': 5,
 'allyears': [2018, 2015, 2006, 1992],
 'type': 'Disease',
 'MeshID': 'D011696',
 'url': 'https://www.ncbi.nlm.nih.gov/mesh/68011696',
 'Summary': 'Any form of purpura in which the PLATELET COUNT is decreased.',
 '_id': 'Disease_Purpura, Thrombocytopenic',
 'datest': 20181201,
 'allwords': ['THROMBOCYTOPENIC PURPURA']}

In [34]:
len(all_es)

11068

In [35]:
noturls=[]
for i in Disease_all:
    if not i.get('url'):
        noturls.append(i)

In [36]:
#13986
len(noturls)

14285

In [37]:
# 17330
len(Disease_all)

17632

In [38]:
json.dump(all_es,open('results/step2/all_es_lack.json','w'))

In [ ]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [ ]:
all_es_result=[]
for j in entity_pairs:
    for i in names[j+'_all']:
        if i.get('specific_type'):
            all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('specific_type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
        else:
            if i.get('type')=='Disease':
                if i.get('url'):
                    all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
            elif i.get('type')=='Mutation':
                if i.get('Genomic View'):
                    all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
            else:
                all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})

In [ ]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [ ]:
collections_pubtator_info = db.pubtator_info

In [ ]:
pubtator_info=list(collections_pubtator_info.find())

In [ ]:
pubtator_info_dict={}
for i in pubtator_info:
    pubtator_info_dict[i.get('PMID')]=i

In [ ]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [ ]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [ ]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [ ]:
jt2if5_brief={}
for i in range(len(jcr)):
    
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

        if jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            iflist.append(jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            iflist.append(jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            iflist.append(0.0)
        if jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            if5list.append(jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            if5list.append(jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            if5list.append(0.0)

In [ ]:
for i in all_es_result:
    iflist=[]
    if5list=[]
    TAlist=[]
    JTlist=[]
    yearlist=[]
    for j in i.get('PMID'):
        if jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            iflist.append(jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            iflist.append(jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            iflist.append(0.0)
        if jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            if5list.append(jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            if5list.append(jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            if5list.append(0.0)
        TAlist.append(pubtator_info_dict.get(j).get('TA'))
        JTlist.append(pubtator_info_dict.get(j).get('JT'))
    for l in i.get('date'):
        yearlist.append(int(str(l)[:4]))
    i['IF']=iflist
    i['IF5']=if5list
    i['TA']=TAlist
    i['JT']=JTlist
    i['year']=yearlist

In [ ]:
json.dump(all_es_result,open('results/step2/all_es_result.json','w'))

In [ ]:
#10907
len(all_es_result)

In [ ]:
blank=[{'Module':'','Description':'','Entity_Relation':'','Username':'','Email':''}]

In [ ]:
json.dump(blank,open('results/step2/feedback.json','w'))

In [1]:
pwd

'/root/hald_autoupdate/2023_03_13'